# 机器学习方法做文本分类：朴素贝叶斯、SVM

## 文本分类=文本表示+模型分类

### 文本表示：BOW/N-gram/TF-IDF/word2vec/word embedding/ELMo
### 分类模型：NB/LR/SVM/LSTM(GRU)/CNN

## 朴素贝叶斯

In [1]:
#准备好数据，我们挑选 科技、汽车、娱乐、军事、运动 总共5类文本数据进行处理。
import jieba
import pandas as pd
df_technology = pd.read_csv("./origin_data/technology_news.csv", encoding='utf-8')
df_technology = df_technology.dropna()

df_car = pd.read_csv("./origin_data/car_news.csv", encoding='utf-8')
df_car = df_car.dropna()

df_entertainment = pd.read_csv("./origin_data/entertainment_news.csv", encoding='utf-8')
df_entertainment = df_entertainment.dropna()

df_military = pd.read_csv("./origin_data/military_news.csv", encoding='utf-8')
df_military = df_military.dropna()

df_sports = pd.read_csv("./origin_data/sports_news.csv", encoding='utf-8')
df_sports = df_sports.dropna()

# 每类数据取20000条
technology = df_technology.content.values.tolist()[1000:21000]
car = df_car.content.values.tolist()[1000:21000]
entertainment = df_entertainment.content.values.tolist()[:20000]
military = df_military.content.values.tolist()[:20000]
sports = df_sports.content.values.tolist()[:20000]

In [2]:
#去除停用词
stopwords=pd.read_csv("origin_data/stopwords.txt",index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values

In [3]:
#对数据做一些预处理，并把处理过后的数据写入新文件夹，避免每次重复操作
def preprocess_text(content_lines, sentences, category, target_path):
    out_f = open(target_path+"/"+category+".txt", 'w')
    for line in content_lines:
        try:
            segs=jieba.lcut(line)
            segs = list(filter(lambda x:len(x)>1, segs)) #没有解析出来的新闻过滤掉
            segs = list(filter(lambda x:x not in stopwords, segs)) #把停用词过滤掉
            sentences.append((" ".join(segs), category))
            out_f.write(" ".join(segs)+"\n")
        except Exception as e:
            print(line)
            continue
    out_f.close()

#生成训练数据
sentences = []
preprocess_text(technology, sentences, 'technology', 'processed_data')
preprocess_text(car, sentences, 'car', 'processed_data')
preprocess_text(entertainment, sentences, 'entertainment', 'processed_data')
preprocess_text(military, sentences, 'military', 'processed_data')
preprocess_text(sports, sentences, 'sports', 'processed_data')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.743 seconds.
Prefix dict has been built successfully.


In [4]:
#打乱一下顺序，生成更可靠的训练集
import random
random.shuffle(sentences)

In [5]:
sentences[:5]

[('丁俊晖 跃龙 中国 德比 中国 第二轮 最受 关注 一场 比赛 此前 两人 2012 国锦赛 强战 交手 丁俊晖 胜利 想到 此前 刚刚 遭遇 胸标 事件 丁俊晖 丝毫 负面影响 开场 火力 全开 109 127 两杆 领先 优势 轻松 横扫 小将 跃龙 晋级 一轮 丁俊晖 对阵 爆冷 击败 奥沙利 乔伊斯',
  'sports'),
 ('媒体报道 网络 照片 显示 首艘 国产 航母 涂装 红色 底漆 岛上 安装 玻璃 相控阵 雷达 设备 曝光 照片 显示 清理 甲板 近期 下水',
  'military'),
 ('李东 百度 地图 AI 赋能 赋能 交互 体验 用户 感受 智能化 个性化 地图 用户 享受 出行 乐趣 享受 地图 人工智能 技术 赋能 整体 信息 架构 地图 信息结构 显得 立体 原本 静态 内容 变得 实时 流动性 用户 出行 提供 智能 精准 解决方案',
  'technology'),
 ('一条 使命 上海 广州 成都 重庆 九大 展开 一系列 媒体 宣传 片中 狗狗 经历 四生 四世 回到 主人 身边 故事 观众 高呼 温暖 影评人 本来 故事 没想到 一点 虐心 暖心 影片 狗狗 陪伴 忠诚 勇气 正义 总有 一个点 触摸 内心深处 柔软 那条 柯基 回家吧',
  'entertainment'),
 ('对此 360 搜索 接口 360 网盾 平台 钓鱼 网站 拦截 日均 拦截 钓鱼 网站 攻击 千万次 截获 新增 钓鱼 网站 千个 预防 不良 钓鱼 网站 搜索 页面 呈现',
  'technology')]

In [6]:
#划分训练集，测试集
from sklearn.model_selection import train_test_split
x, y = zip(*sentences)
print(len(y))
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1234)

87595


In [7]:
len(x_train)

65696

In [8]:
#对文本抽取词袋模型特征
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams
    max_features=4000,  # keep the most common 4000 ngrams
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

In [9]:
#分类器
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [10]:
#测试集效果
classifier.score(vec.transform(x_test), y_test)

0.8368875291109183

In [11]:
len(x_test)

21899

In [12]:
#加入抽取2-gram和3-gram的统计特征，把词库的量放大一点
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(
    analyzer='word', # tokenise by character ngrams
    ngram_range=(1,4),  # use ngrams of size 1, 2, 3, 4
    max_features=20000,  # keep the most common 2000 ngrams
)
vec.fit(x_train)

def get_features(x):
    vec.transform(x)

In [13]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB()
classifier.fit(vec.transform(x_train), y_train)
classifier.score(vec.transform(x_test), y_test)

0.8761130645234942

### 交叉验证

In [14]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score
import numpy as np

def stratifiedkfold_cv(x, y, clf_class, shuffle=True, n_folds=5, **kwargs):
    stratifiedk_fold = StratifiedKFold(n_splits=n_folds, shuffle=shuffle)
    y_pred = y[:]
    for train_index, test_index in stratifiedk_fold.split(x, y):
        X_train, X_test = x[train_index], x[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
    return y_pred 

NB = MultinomialNB
print(precision_score(y, stratifiedkfold_cv(vec.transform(x),np.array(y),NB), average='macro'))

0.8810872212226274


In [15]:
#做成一个类
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB


class TextClassifier():

    def __init__(self, classifier=MultinomialNB()):
        self.classifier = classifier
        self.vectorizer = CountVectorizer(analyzer='word', ngram_range=(1,4), max_features=20000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

    def save_model(self, path):
        dump((self.classifier, self.vectorizer), path)

    def load_model(self, path):
        self.classifier, self.vectorizer = load(path)

In [16]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.8761130645234942


## SVM文本分类

In [17]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

0.8465683364537193

In [18]:
#rb核
from sklearn.svm import SVC
svm = SVC()
svm.fit(vec.transform(x_train), y_train)
svm.score(vec.transform(x_test), y_test)

0.8479839262066761

In [19]:
## 使用tf-idf作为特征
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC


class TextClassifier():

    def __init__(self, classifier=SVC(kernel='linear')):
        self.classifier = classifier
        self.vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,3), max_features=12000)

    def features(self, X):
        return self.vectorizer.transform(X)

    def fit(self, X, y):
        self.vectorizer.fit(X)
        self.classifier.fit(self.features(X), y)

    def predict(self, x):
        return self.classifier.predict(self.features([x]))

    def score(self, X, y):
        return self.classifier.score(self.features(X), y)

In [20]:
text_classifier = TextClassifier()
text_classifier.fit(x_train, y_train)
print(text_classifier.predict('这 是 有史以来 最 大 的 一 次 军舰 演习'))
print(text_classifier.score(x_test, y_test))

['military']
0.877345997534134
